In [1]:
import requests
import json
import random 

URL = "http://localhost:"
ports = range(10000,10006)
def random_url():
    return (URL + str(random.choice(ports))+"/")

def address(addr):
    return str(addr).rjust(16, '0')


### View initial state

In [2]:
initial_utxo = requests.get(random_url() + "utxos/" + address(0)).content.decode('utf-8')
print(f"INITIAL UTXO\n {initial_utxo}")

INITIAL UTXO
 {
  "utxos": [{
    "txId": "0x00000000001",
    "utxoId": "00000000-0000-0000-0000-000000000000",
    "addr": "0000000000000000",
    "coins": "9223372036854775807"
  }]
}


### Send money between addresses handled by different shards


In [3]:
# from 0 to 1
requests.post(random_url() + "sendCoins", params={"to": address(1), "from": address(0), "coins": "100"})
# from 1 to 2
requests.post(random_url() + "sendCoins", params={"to": address(2), "from": address(1), "coins": "90"})
# from 2 to 3
requests.post(random_url() + "sendCoins", params={"to": address(3), "from": address(2), "coins": "80"})
# from 3 to 4
requests.post(random_url() + "sendCoins", params={"to": address(4), "from": address(3), "coins": "70"})
# from 4 to 1
requests.post(random_url() + "sendCoins", params={"to": address(1), "from": address(4), "coins": "60"})

<Response [200]>

### Get UTxOs after send money series

In [4]:
for addr in range(1, 5):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")


1 UTxOs

{
  "utxos": [{
    "txId": "f3a3b16c-3358-4f3f-a470-cefb05237585",
    "utxoId": "cab1bdaa-6f73-3c3f-b680-2c0dcade1404",
    "addr": "0000000000000001",
    "coins": "10"
  }, {
    "txId": "f0b16850-4485-49e8-a593-2899fee2d3a8",
    "utxoId": "5e1481d1-975a-3bdb-a2c6-847511550a66",
    "addr": "0000000000000001",
    "coins": "60"
  }]
}

2 UTxOs

{
  "utxos": [{
    "txId": "7f1e7d4b-a395-41e5-a503-01d2f7865ab8",
    "utxoId": "e7bd638c-5759-3971-874d-2935aa75ce53",
    "addr": "0000000000000002",
    "coins": "10"
  }]
}

3 UTxOs

{
  "utxos": [{
    "txId": "1307c100-7578-4523-8101-68998fcfcdeb",
    "utxoId": "e845e4c1-0c3b-304d-853b-08ec48e9c571",
    "addr": "0000000000000003",
    "coins": "10"
  }]
}

4 UTxOs

{
  "utxos": [{
    "txId": "f0b16850-4485-49e8-a593-2899fee2d3a8",
    "utxoId": "001c6fd4-2b55-392e-995a-391ec6d7cdb2",
    "addr": "0000000000000004",
    "coins": "10"
  }]
}


### Get ledger history for address 0

In [5]:
ledger = requests.get(random_url() + "ledger/" + address(0)).content.decode('utf-8')
print(f"Ledger for 0 \n{ledger}")

Ledger for 0 
This is the history for address 0000000000000000:
{
  "txs": [{
    "txId": "44b5b8e1-ab59-4f17-9d53-f813d9e58337",
    "inputs": [{
      "txId": "0x00000000001",
      "utxoId": "00000000-0000-0000-0000-000000000000",
      "addr": "0000000000000000",
      "coins": "9223372036854775807"
    }],
    "outputs": [{
      "addr": "0000000000000001",
      "coins": "100"
    }, {
      "addr": "0000000000000000",
      "coins": "9223372036854775707"
    }],
    "timestamp": "2022-01-26T16:03:55.909Z"
  }]
}


### Get entire ledger history

In [6]:
ledger = requests.get(random_url() + "ledger/").content.decode('utf-8')
print(f"{ledger}")

This is the entire ledger history at the moment:
{
  "txs": [{
    "txId": "f0b16850-4485-49e8-a593-2899fee2d3a8",
    "inputs": [{
      "txId": "1307c100-7578-4523-8101-68998fcfcdeb",
      "utxoId": "a3965b74-2542-3f4c-96c8-fe407ee08fab",
      "addr": "0000000000000004",
      "coins": "70"
    }],
    "outputs": [{
      "addr": "0000000000000001",
      "coins": "60"
    }, {
      "addr": "0000000000000004",
      "coins": "10"
    }],
    "timestamp": "2022-01-26T16:03:58.703Z"
  }, {
    "txId": "1307c100-7578-4523-8101-68998fcfcdeb",
    "inputs": [{
      "txId": "7f1e7d4b-a395-41e5-a503-01d2f7865ab8",
      "utxoId": "10718417-a84e-31cc-a53a-7b50942232db",
      "addr": "0000000000000003",
      "coins": "80"
    }],
    "outputs": [{
      "addr": "0000000000000004",
      "coins": "70"
    }, {
      "addr": "0000000000000003",
      "coins": "10"
    }],
    "timestamp": "2022-01-26T16:03:58.215Z"
  }, {
    "txId": "7f1e7d4b-a395-41e5-a503-01d2f7865ab8",
    "inputs": [

### Make a transaction

Transfer half of the coins for address 1 to address 0

In [7]:
utxos_1 = json.loads(requests.get(random_url() + "utxos/" + address(0)).content.decode('utf-8'))
coins = sum([int(utxo['coins']) for utxo in utxos_1['utxos']]) 
half_coins = coins //2
output1 = {"addr": address(1), "coins": str(half_coins)}
output2 = {"addr": address(0), "coins": str(coins-half_coins)}
new_tx = {}
new_tx['inputs'] = utxos_1['utxos']
new_tx['outputs'] = output1,output2
new_tx=json.dumps(new_tx).encode('utf-8')
res = requests.post(random_url() + "submitTx", data=new_tx, headers={'Content-Type': 'application/json'})
print(res.content.decode('utf-8'))

Submitted successfully with ID bfb3bc3f-f322-4bf3-8b76-c37cf03e632e 


### Print updated UTxOs for 0 and 1 after the transaction

In [8]:
for addr in range(0, 2):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")


0 UTxOs

{
  "utxos": [{
    "txId": "bfb3bc3f-f322-4bf3-8b76-c37cf03e632e",
    "utxoId": "aa9eba29-2ae1-3801-ade0-44c633230c1b",
    "addr": "0000000000000000",
    "coins": "4611686018427387854"
  }]
}

1 UTxOs

{
  "utxos": [{
    "txId": "f3a3b16c-3358-4f3f-a470-cefb05237585",
    "utxoId": "cab1bdaa-6f73-3c3f-b680-2c0dcade1404",
    "addr": "0000000000000001",
    "coins": "10"
  }, {
    "txId": "f0b16850-4485-49e8-a593-2899fee2d3a8",
    "utxoId": "5e1481d1-975a-3bdb-a2c6-847511550a66",
    "addr": "0000000000000001",
    "coins": "60"
  }, {
    "txId": "bfb3bc3f-f322-4bf3-8b76-c37cf03e632e",
    "utxoId": "d07e7895-351f-3a54-8ddf-075b8c52f489",
    "addr": "0000000000000001",
    "coins": "4611686018427387853"
  }]
}
